# Tutorial Simplex
This is based on this [link.](https://www.softcover.io/read/7b8eb7d0/juliabook/simplex)

In [26]:
using Pkg
Pkg.add("Clp")
Pkg.add("JuMP")
Pkg.add("Cbc");
Pkg.add("MathProgBase");
Pkg.add("Combinatorics");
using StatsPlots, Plots
using Combinatorics, LinearAlgebra

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [fdba3010] + MathProgBase v0.7.8
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


### **1.** Searching all Basic Feasible Solutions

In [20]:
function isnonnegative(x::Array{Float64, 1})
  return length( x[ x .< 0] ) == 0
end

function searchBFS(c, A, b)
  m, n = size(A)
  @assert rank(A) == m

  opt_x = zeros(n)
  obj = Inf

  for b_idx in combinations(1:n, m)
    B = A[:, b_idx]
    c_B = c[b_idx]
    x_B = inv(B) * b

    if isnonnegative(x_B)
      z = dot(c_B, x_B)
      if z < obj
        obj = z
        opt_x = zeros(n)
        opt_x[b_idx] = x_B
      end
    end

    println("Basis:", b_idx)
    println("\t x_B = ", x_B)
    println("\t nonnegative? ", isnonnegative(x_B))
    if isnonnegative(x_B)
      println("\t obj = ", dot(c_B, x_B))
    end

  end

  return opt_x, obj
end

searchBFS (generic function with 1 method)

In [24]:
c = [-3; -2; -1; -5; 0; 0; 0]
A = [7 3 4 1 1 0 0 ;
     2 1 1 5 0 1 0 ;
     1 4 5 2 0 0 1 ]
b = [7; 3; 8]
opt_x, obj = searchBFS(c, A, b)

Basis:[1, 2, 3]
	 x_B = [0.25, 4.75, -2.25]
	 nonnegative? false
Basis:[1, 2, 4]
	 x_B = [0.172414, 1.87931, 0.155172]
	 nonnegative? true
	 obj = -5.051724137931034
Basis:[1, 2, 5]
	 x_B = [0.571429, 1.85714, -2.57143]
	 nonnegative? false
Basis:[1, 2, 6]
	 x_B = [0.16, 1.96, 0.72]
	 nonnegative? true
	 obj = -4.3999999999999995
Basis:[1, 2, 7]
	 x_B = [-2.0, 7.0, -18.0]
	 nonnegative? false
Basis:[1, 3, 4]
	 x_B = [0.121622, 1.47297, 0.256757]
	 nonnegative? true
	 obj = -3.121621621621621
Basis:[1, 3, 5]
	 x_B = [0.777778, 1.44444, -4.22222]
	 nonnegative? false
Basis:[1, 3, 6]
	 x_B = [0.0967742, 1.58065, 1.22581]
	 nonnegative? true
	 obj = -1.8709677419354835
Basis:[1, 3, 7]
	 x_B = [5.0, -7.0, 38.0]
	 nonnegative? false
Basis:[1, 4, 5]
	 x_B = [34.0, -13.0, -218.0]
	 nonnegative? false
Basis:[1, 4, 6]
	 x_B = [0.461538, 3.76923, -16.7692]
	 nonnegative? false
Basis:[1, 4, 7]
	 x_B = [0.969697, 0.212121, 6.60606]
	 nonnegative? true
	 obj = -3.9696969696969693
Basis:[1, 5, 6]
	 x

([0.172414, 1.87931, 0.0, 0.155172, 0.0, 0.0, 0.0], -5.051724137931034)

### **2.** Using solvers to solve the LP problem

In [38]:
using JuMP, Clp

┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1192


In [59]:
Af = convert(Array{Float64}, A)
cf = convert(Array{Float64}, c)
bf = convert(Array{Float64}, b)

sol = linprog(cf, Af, '=', bf, ClpSolver()) #Only works with Float instead of Int
sol.status, sol.objval, sol.sol

(:Optimal, -5.051724137931035, [0.172414, 1.87931, 0.0, 0.155172, 0.0, 0.0, 0.0])